<a href="https://colab.research.google.com/github/beyg1/Q4/blob/main/Quiz%20Practice/verbose_Agent_Runner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uq openai-agents

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.1/161.1 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.5/158.5 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.0 MB/s eta 0:00:00


In [2]:
from google.colab import userdata

GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

In [4]:
import nest_asyncio
nest_asyncio.apply()
#Agent loop doesnt run if dont make notebook capable of running async,
#because of AsyncOpenAI

In [5]:
from openai import AsyncOpenAI
from agents import Agent, Runner, OpenAIChatCompletionsModel, enable_verbose_stdout_logging, set_tracing_disabled, function_tool

set_tracing_disabled(True)
enable_verbose_stdout_logging()

external_client = AsyncOpenAI(
    api_key=GEMINI_API_KEY,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

model = OpenAIChatCompletionsModel(
    model = "gemini-2.5-flash-lite-preview-06-17",
    openai_client = external_client
)

We can send 4 things to LLM.
1. System Prompt
2. User Prompt
3. Tool Schema
4. Tool Message

In [ ]:
# Error Case: No model settings provided since it's using external LLM
agent = Agent(
    name="Assistant",
    instructions="You are a helpful assistant.",
    #model=model  openai agents by default push for gpt-4o model no matter which LLM api you choose, so if you don't choose a model it will go for gpt-4o
    model = "gemini-2.0-flash", # this will give error too since openai_client is not set and can't pass that directly in Agent

)
response = Runner.run_sync(agent, "What is the meaning of life? answer in one sentence")
print(response.final_output)

In [10]:
# Working Case : Verbose enabled with simple calls
agent = Agent(
    name="Assistant",
    instructions="You are a helpful assistant.",
    model=model
)
response = Runner.run_sync(agent, "Hey, What's the weather in lahore")
print(response.final_output)


Tracing is disabled. Not creating trace Agent workflow


DEBUG:openai.agents:Tracing is disabled. Not creating trace Agent workflow


Setting current trace: no-op


DEBUG:openai.agents:Setting current trace: no-op


Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x7c75b3f133b0>


...

LLM resp:
{
  "content": "The weather in Lahore is currently **28°C** and it is **partly cloudy**. \n\nThe wind is blowing from the **West** at **9 km/h**. \n\nThe humidity is **69%**.\n\nFor the rest of the day, the forecast is for **partly cloudy skies** with a high of **32°C** and a low of **20°C**.",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}



DEBUG:openai.agents:LLM resp:
{
  "content": "The weather in Lahore is currently **28°C** and it is **partly cloudy**. \n\nThe wind is blowing from the **West** at **9 km/h**. \n\nThe humidity is **69%**.\n\nFor the rest of the day, the forecast is for **partly cloudy skies** with a high of **32°C** and a low of **20°C**.",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}



Resetting current trace


DEBUG:openai.agents:Resetting current trace


The weather in Lahore is currently **28°C** and it is **partly cloudy**. 

The wind is blowing from the **West** at **9 km/h**. 

The humidity is **69%**.

For the rest of the day, the forecast is for **partly cloudy skies** with a high of **32°C** and a low of **20°C**.


In [9]:
# Working Case : Verbose enabled with function calls
@function_tool
def get_current_weather(location: str) -> str:
   """Gets the current weather in a given location"""
   return f"The current weather in {location} is Sunny"

agent = Agent(
    name="Assistant",
    instructions="You are a helpful assistant.",
    model=model,
    tools=[get_current_weather]
)
response = Runner.run_sync(agent, "Hey, What's the weather in lahore")
print(response.final_output)

Tracing is disabled. Not creating trace Agent workflow


DEBUG:openai.agents:Tracing is disabled. Not creating trace Agent workflow


Setting current trace: no-op


DEBUG:openai.agents:Setting current trace: no-op


Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x7c75b3f63bf0>


...

LLM resp:
{
  "content": "The current weather in lahore is Sunny.",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}



DEBUG:openai.agents:LLM resp:
{
  "content": "The current weather in lahore is Sunny.",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}



Resetting current trace


DEBUG:openai.agents:Resetting current trace


The current weather in lahore is Sunny.
